In [179]:
from django.contrib.auth.models import User
from myboard import models
from django.db import connection  # db를 연계할 수 있음.
from django.shortcuts import render

In [5]:
data = User.objects.all()

In [6]:
# db에 대하 정보를 알 수 있음.
models.Board??

In [8]:
#  내가 사용한 SQL 문에 대한 history
connection.queries[-1]

{'sql': 'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" LIMIT 21',
 'time': '0.000'}

## SQL 문 실행

In [9]:
User.objects.raw('SELECT max(age) FROM myapp_user')

<RawQuerySet: SELECT max(age) FROM myapp_user>

In [14]:
# 위의 문장보다 아래 방법 권장
# cursor: sql문을 관리하는 객체 -> 어디를 가리키고 있는지 표시(어떤 row)
cursor = connection.cursor()
cursor.execute('SELECT max(age) from myapp_user')

# cursor를 fetch해야 함.(db에서 검색된 결과가 굉장히 많을 수 있으므로 얼만큼 갖고올지 확인)
max_value = cursor.fetchone()[0] 
print(max_value)

123


In [20]:
cursor = connection.cursor()
cursor.execute('SELECT * from myboard_board')
results = cursor.fetchall()  # 데이터가 많지 않을 때만 수행
print(type(results), type(results[0]))
print(results[0])

<class 'list'> <class 'tuple'>
(5, 'kjkjkkjkjk', 'kkjkjkjkjkj', datetime.datetime(2020, 2, 19, 8, 22, 23, 468352), None, 0, None, 'data', 2)


In [21]:
for i in results:
    print(i)

(5, 'kjkjkkjkjk', 'kkjkjkjkjkj', datetime.datetime(2020, 2, 19, 8, 22, 23, 468352), None, 0, None, 'data', 2)
(8, 'hjhkjhj', 'hjhkhhjkhkhj', datetime.datetime(2020, 2, 20, 1, 28, 34, 298647), None, 1, None, 'common', 2)
(9, 'hjhjkhjh', 'jhkhjhjhjkhjkh', datetime.datetime(2020, 2, 20, 1, 28, 48, 697712), None, 0, None, 'etc', 2)
(10, 'kjkjkjkjkjkjk', 'kjkjkjkjkjkjkjk', datetime.datetime(2020, 2, 20, 1, 29, 1, 882260), None, 0, None, 'data', 2)
(11, 'kjkjkjkjkj', 'jkjkjkjkjkjkjk', datetime.datetime(2020, 2, 20, 1, 36, 51, 908939), None, 0, None, 'common', 2)
(12, '추가하자', '너라넝라ㅓㄴㅇ러나이ㅓ', datetime.datetime(2020, 2, 20, 7, 44, 3, 880978), None, 0, None, 'common', 2)
(13, 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 'ㅓㅣㅏㅓㅏㅣㅓㅣㅓㅏㅓㅏㅣ', datetime.datetime(2020, 2, 20, 7, 44, 15, 712812), None, 0, None, 'common', 2)
(14, 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 'ㅏㅓㅏㅓㅣㅏㅓㅏㅣㅓㅏㅣㅓㅣㅓㅣ', datetime.datetime(2020, 2, 20, 7, 44, 21, 839611), None, 0, None, 'common', 2)
(15, 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 'ㅏㅓㅏㅣㅓㅏㅣㅓㅏㅣㅓㅏㅣㅓㅏㅣ', datetime.datetime(2020, 2, 20, 7, 44, 36, 63932), None, 0

### \*의 의미
- 모든 필드를 의미
- 보안에 취약
- 네트워크 트래픽 유발
- 일반적으로 *는 쓰지 말라고 함.

In [23]:
cursor = connection.cursor()
cursor.execute('SELECT id, title, cnt from myboard_board')
results = cursor.fetchall()  # 데이터가 많지 않을 때만 수행
print(results[0])

(5, 'kjkjkkjkjk', 0)


In [24]:
cursor = connection.cursor()
cursor.execute('SELECT id, title, category from myboard_board')
results = cursor.fetchall()  # 데이터가 많지 않을 때만 수행
print(results[0])

(5, 'kjkjkkjkjk', 'data')


## 1. 특정 유저가 쓴 글 검색

- board에는 저자정보가 아닌 id번호만 들어가있음.

### 1.1. 조건절에 다른 테이블 정보로 slect


In [34]:
sql = '''
select id, title, cnt
from myboard_board
where author_id = (
select id from auth_user where username = 'lee')
'''

cursor = connection.cursor()
cursor.execute(sql)
results = cursor.fetchall()  
print(results)

[(5, 'kjkjkkjkjk', 0), (8, 'hjhkjhj', 1), (9, 'hjhjkhjh', 0), (10, 'kjkjkjkjkjkjk', 0), (11, 'kjkjkjkjkj', 0), (12, '추가하자', 0), (13, 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 0), (14, 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 0), (15, 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 0)]


### 1.2. join 사용하기
- 위의 방법은...board table에 use 정보가 없기 때문에, user정보를 표현할 수 없음.
- join으로 해결

In [35]:
sql = '''
select b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id
'''

cursor = connection.cursor()
cursor.execute(sql)
results = cursor.fetchall()  
print(results)

[(5, 'kjkjkkjkjk', 0, 'lee'), (8, 'hjhkjhj', 1, 'lee'), (9, 'hjhjkhjh', 0, 'lee'), (10, 'kjkjkjkjkjkjk', 0, 'lee'), (11, 'kjkjkjkjkj', 0, 'lee'), (12, '추가하자', 0, 'lee'), (13, 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 0, 'lee'), (14, 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 0, 'lee'), (15, 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 0, 'lee')]


In [36]:
sql = '''
select b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='lee'
'''

cursor = connection.cursor()
cursor.execute(sql)
results = cursor.fetchall()  
print(results)

[(5, 'kjkjkkjkjk', 0, 'lee'), (8, 'hjhkjhj', 1, 'lee'), (9, 'hjhjkhjh', 0, 'lee'), (10, 'kjkjkjkjkjkjk', 0, 'lee'), (11, 'kjkjkjkjkj', 0, 'lee'), (12, '추가하자', 0, 'lee'), (13, 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 0, 'lee'), (14, 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 0, 'lee'), (15, 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 0, 'lee')]


## 2. cursor의 개념
- 전체 레코드에서 어디를 읽었는지 나타내는 개념
- 모든 레코드를 읽고나면 fetch는 None.
- 읽은 데이터를 보관하지 않고,다 읽은 후, 처음을 돌아가지 않음.

In [38]:
cursor = connection.cursor()
cursor.execute(sql)
r = cursor.fetchone()
print(r)

(5, 'kjkjkkjkjk', 0, 'lee')


In [39]:
r = cursor.fetchone()
print(r)

(8, 'hjhkjhj', 1, 'lee')


In [43]:
r = cursor.fetchone()
len(r)   

4

- 전체 data 6개중 fetchone을 2번 실행했기 때문에
- 나머지 4개 데이터만 갖고있음.
- fecthall을 하고 나면 반환한 레코드가 없으므로 None을 return

### 2.1 전체 데이터 가져오기

In [48]:
sql = '''
select b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='lee'
'''

cursor = connection.cursor()
cursor.execute(sql)
results = cursor.fetchall()  

for r in results:
    print(r)

(5, 'kjkjkkjkjk', 0, 'lee')
(8, 'hjhkjhj', 1, 'lee')
(9, 'hjhjkhjh', 0, 'lee')
(10, 'kjkjkjkjkjkjk', 0, 'lee')
(11, 'kjkjkjkjkj', 0, 'lee')
(12, '추가하자', 0, 'lee')
(13, 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 0, 'lee')
(14, 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 0, 'lee')
(15, 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 0, 'lee')


##  3.  특정 filed값을 갖고오기

- 컬럼명으로 접근할 수 없음...


In [52]:
for r in results:
    print(r[1])

kjkjkkjkjk
hjhkjhj
hjhjkhjh
kjkjkjkjkjkjk
kjkjkjkjkj
추가하자
ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ
ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ
ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ


### 3.1. 파이썬으로 dict로 변환하기

In [56]:
# 각 걸럼에 대한 정보를 알 수 있음.(내가 요청한 데이터에 대해서)
cursor.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

In [58]:
d = dict(zip(['id', 'title', 'cnt', 'username'], results[0]))
print(d)

{'id': 5, 'title': 'kjkjkkjkjk', 'cnt': 0, 'username': 'lee'}


In [60]:
def dectfetchall(cursor):
    desc = cursor.description
    return[
        dict(zip([col[0] for col in desc], row))
        for row in cursor.fetchall()
    ]
    

In [64]:
cursor = connection.cursor()
cursor.execute('SELECT * from myboard_board')
d = dectfetchall(cursor)
print(d)


[{'id': 5, 'title': 'kjkjkkjkjk', 'text': 'kkjkjkjkjkj', 'created_data': datetime.datetime(2020, 2, 19, 8, 22, 23, 468352), 'published_date': None, 'cnt': 0, 'image': None, 'category': 'data', 'author_id': 2}, {'id': 8, 'title': 'hjhkjhj', 'text': 'hjhkhhjkhkhj', 'created_data': datetime.datetime(2020, 2, 20, 1, 28, 34, 298647), 'published_date': None, 'cnt': 1, 'image': None, 'category': 'common', 'author_id': 2}, {'id': 9, 'title': 'hjhjkhjh', 'text': 'jhkhjhjhjkhjkh', 'created_data': datetime.datetime(2020, 2, 20, 1, 28, 48, 697712), 'published_date': None, 'cnt': 0, 'image': None, 'category': 'etc', 'author_id': 2}, {'id': 10, 'title': 'kjkjkjkjkjkjk', 'text': 'kjkjkjkjkjkjkjk', 'created_data': datetime.datetime(2020, 2, 20, 1, 29, 1, 882260), 'published_date': None, 'cnt': 0, 'image': None, 'category': 'data', 'author_id': 2}, {'id': 11, 'title': 'kjkjkjkjkj', 'text': 'jkjkjkjkjkjkjk', 'created_data': datetime.datetime(2020, 2, 20, 1, 36, 51, 908939), 'published_date': None, 'cnt'

## 4. web에서 바로 하지 말고...여기서 연습하기

In [ ]:
from django.test import RequestFactory
rf= RequestFactory()

In [100]:
# username = request.session.get('username', '')
username = 'lee'
user = User.objects.all().get(username=username )
print(user)

lee


In [104]:
context = models.Board.objects.all().filter(author = user,
                                  category = category)
context

<QuerySet [<Board: hjhkjhj>, <Board: kjkjkjkjkj>, <Board: 추가하자>, <Board: ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ>, <Board: ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ>, <Board: ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ>]>

In [132]:
def listsql(request, category, page):
    page = request.GET.get('page', 1)
    category = request.GET.get('category', 'common')
    # username = request.session.get('username', '')
    username = 'lee'
    user = User.objects.all().get(username=username )
    datas = models.Board.objects.all().filter(author = user,
                                                  category = category)
    page = int(page)
    subs = datas[(page-1)*3: (page*3)]
    print('subs:',subs)
    datas = {'datas': [{'username': data.author, 'title': data.title, 'cnt': data.cnt} for data in subs]}

#     return render(request, apps.APP + '/listsql.html', context)
    return print(datas)

In [133]:
url = '/?category=common&page=2'
request = rf.get(url)

In [134]:
page = request.GET.get('page', 1)
print(page)
category = request.GET.get('category', 'common')
print(category)


2
common


In [135]:
listsql(request, category, page)

subs: <QuerySet [<Board: ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ>, <Board: ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ>, <Board: ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ>]>
{'datas': [{'username': <User: lee>, 'title': 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 'cnt': 0}, {'username': <User: lee>, 'title': 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 'cnt': 0}, {'username': <User: lee>, 'title': 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 'cnt': 0}]}


## 5. connection사용하기 

In [180]:
sql = '''
select b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='lee' and category='common'
'''

cursor = connection.cursor()
cursor.execute(sql)


In [181]:
desc = cursor.description
desc

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

In [182]:
def dictfetchall(cursor):
    desc = cursor.description
    return[
        dict(zip([col[0] for col in desc], row))
        for row in cursor.fetchall()
    ]

In [183]:
d = dictfetchall(cursor)
print(d)

[{'id': 8, 'title': 'hjhkjhj', 'cnt': 1, 'username': 'lee'}, {'id': 11, 'title': 'kjkjkjkjkj', 'cnt': 0, 'username': 'lee'}, {'id': 12, 'title': '추가하자', 'cnt': 0, 'username': 'lee'}, {'id': 13, 'title': 'ㅏㅓㅏㅓㅣㅓㅏㅣㅓㅣㅏ', 'cnt': 0, 'username': 'lee'}, {'id': 14, 'title': 'ㅏㅓㅣㅏㅓㅏㅓㅏㅓㅣ', 'cnt': 0, 'username': 'lee'}, {'id': 15, 'title': 'ㅏㅓㅣㅓㅏㅓㅏㅓㅏㅓㅏ', 'cnt': 0, 'username': 'lee'}]


In [184]:
def listsql(request, category, page):
    page = request.GET.get('page', 1)
    category = request.GET.get('category', 'common')
    # username = request.session.get('username', '')
    username = 'lee'
    user = User.objects.all().get(username=username )
    datas = models.Board.objects.all().filter(author = user,
                                                  category = category)
    page = int(page)
    subs = datas[(page-1)*3: (page*3)]
    print('subs:',subs)
    datas = {'datas': [{'username': data.author, 'title': data.title, 'cnt': data.cnt} for data in subs]}

#     return render(request, apps.APP + '/listsql.html', context)
    return print(datas)

In [193]:
def listsql(request, category, page):
    page = request.GET.get('page', 1)
    category = request.GET.get('category', 'common')
    username = 'lee'
    user = User.objects.all().get(username=username )
    
    sql = f"""
        select b.id, title, cnt,username,category 
       from myboard_board b, auth_user u 
       where b.author_id = u.id and username = '{username}' and category = '{category}'
       """
    

    cursor = connection.cursor()
    cursor.execute(sql)
    datas = dictfetchall(cursor)

    page = int(page)
    subs = datas[(page-1)*3: (page*3)]
    datas = {'datas': subs}

    return  render(request,  'myboard/listsql.html', datas) 


In [199]:
listsql(request, category, page)

<HttpResponse status_code=200, "text/html; charset=utf-8">

In [198]:
from myboard import views
response =listsql(request, 'common', 1)
print(response.content.decode())